In [ ]:
wine

In [ ]:
from IPython.display import Image
import pydotplus
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz

# Carregar o conjunto de dados Wine
wine = datasets.load_wine()
X = wine.data
y = wine.target

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Criar o modelo de árvore de decisão
clf = DecisionTreeClassifier(random_state=42)

# Treinar o modelo
clf.fit(X_train, y_train)

# Gerar o código DOT para visualizar a árvore de decisão
dot_data = export_graphviz(clf, out_file=None,
                           feature_names=wine.feature_names,
                           class_names=wine.target_names,
                           filled=True, rounded=True,
                           special_characters=True)

# Criar o gráfico da árvore de decisão
graph = pydotplus.graph_from_dot_data(dot_data)

# Salvar a imagem em formato PNG
graph.write_png('wine_tree.png')

# Mostrar a imagem da árvore de decisão
Image(graph.create_png())